In [0]:
# This script helps convert from UTC time to local time
# First download a dump of cities and timezones from http://download.geonames.org/export/dump/, e.g. cities1000.txt
# This example based on the US

# First set up a dict of cities and timezones
from collections import defaultdict

filename = 'cities1000.txt' # or cities15000.txt

# -- Google Colab
from google.colab import files
import pandas as pd
import io
print("Upload " + filename + ":")
files.upload()
# --

file = open(filename,"r")

city2tz = defaultdict(set)
for line in file:
    fields = line.split('\t')
    if fields:
        name, asciiname, alternatenames = fields[1:4]
        state = fields[10]
        timezone = fields[-2].strip()
        if timezone and "America" in timezone:
            for city in [name, asciiname] + alternatenames.split(','):
                city = city.strip().lower()
                if city:
                    city2tz[city].add((state,timezone))
                    
print("Number of available city names (with aliases): %d" % len(city2tz))

n = sum((len(timezones) > 1) for city, timezones in city2tz.items())
print("")
print("Find number of ambigious city names\n "
      "(that have more than one associated timezone): %d" % n)

Upload cities15000.txt:


Saving cities15000.txt to cities15000.txt
Number of available city names (with aliases): 60038

Find number of ambigious city names
 (that have more than one associated timezone): 3155


In [0]:
# 1) Now can e.g. look for a specific city; note that there are often matching cities in multiple US states
from datetime import datetime
import pytz
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
city = "Austin".lower()
for tzname in city2tz[city]:
    now = datetime.now(pytz.timezone(tzname[1]))
    print("")
    print("%s is in %s timezone" % (city, tzname[1]))
    print("Current time in %s, %s is %s" % (city, tzname[0], now.strftime(fmt)))
    tz = tzname


austin is in America/New_York timezone
Current time in austin, OH is 2019-12-15 21:17:13 EST-0500

austin is in America/Chicago timezone
Current time in austin, TX is 2019-12-15 20:17:13 CST-0600

austin is in America/Chicago timezone
Current time in austin, MN is 2019-12-15 20:17:13 CST-0600


In [0]:
# 2) Or, paste in a data set that you have copied to the clipboard e.g. from Google Sheets 
# In this example the dates are in a Salesforce format (but you can edit)
import pandas as pd

df = pd.read_clipboard()
df.head()

In [0]:
import re

hours = [0] * 24
timezones = []
local_times = []

# Loop through your data and look for matching city/state combinations
for i, row in df.iterrows():
    city = row['City'].lower()
    state = row['State']
    date = row['UTC Date']
    match = False
    for tz in city2tz[city]:
        if tz[0] == state:
            print("%s, %s is in %s timezone" % (city, tz[0], tz[1]))
            groups = re.match("(\d\d\d\d)-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).*",date).groups()
            utc_time = datetime(int(groups[0]), int(groups[1]), int(groups[2]), 
                                int(groups[3]), int(groups[4]), int(groups[5]))
            local_datetime = pytz.utc.localize(utc_time).astimezone(pytz.timezone(tz[1]))
            print("%s in UTC is equivalent to %s in local time\n" % (date,local_datetime))
            match = True
    
    # Append matches, otherwise append empty result
    if match:
        timezones.append(tz[1])
        local_times.append(local_datetime)
        local_hours.append(local_hour)
    else: 
        print(city + " does not have a match\n")
        timezones.append('')
        local_times.append('')
        local_hours.append('')

In [0]:
# Add the results to your original dataframe
df['Timezone'] = timezones
df['Local Time'] = local_times
df.head()

In [0]:
# Copy output ready to paste back into e.g. Google Sheets
df.to_clipboard(index=False)